In [1]:
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# #from selenium.webdriver.support import expected_conditions
# from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.common.action_chains import ActionChains
# # from scraper_fonctions import defilement, recolte_resultats, click_volet_telechargement, verif_bouton_txt, recolte_metadonnees, recolte_metadonnees_générique, recolte_texte
from sklearn.feature_extraction.text import TfidfVectorizer

#import time
#import webbrowser
import pandas as pd
#import requests
#import csv
import re
# import matplotlib.pyplot as plt
# import seaborn as sns

df = pd.read_csv("data/test.csv")

In [2]:
def suppr_espace(text):
        # \s = any whitespace | + = one or more
        regex = re.compile(r"\s+")
        regex
        text = re.sub(regex, "", str(text))
        text
        return text


In [3]:
def recup_nom_auteur(text):
    try:
        regex = re.compile(r",")
        index_auteur = re.search(regex, text)
        
        return text[:index_auteur.end()-1]
    except AttributeError:
        return text

In [4]:
# Filtre pour effacer l'entête

def efface_entete(text):
    try:
        regex = re.compile(r"Le taux de reconnaissance estimé pour ce document est de [0-9]{2,}%.")
        fin_entete = re.search(regex, str(text)).end()
    except AttributeError:
        # Normalement, doit récupérer le document Livre/Manuscrit/Gallica sans en-tête OCR avec % réussite
        return text
    
    regex = re.compile(r"En savoir plus sur l\'OCR")
    text = re.sub(regex, "", text)
    
    return text[fin_entete:]

In [5]:
def retire_saut_ligne(text):
    regex = re.compile(r"\n")
    text = re.sub(regex, " ", str(text))
    
    return text

In [6]:
# # Applique la fonction suppr_espace à toute la colonne

# df["date"] = df["date"].apply(suppr_espace)


# # Compte le nombre d'occurrence de chaque date unique
# liste_dates = df[df["date"].str.contains(r"^[0-9]{4}", regex=True)]["date"].value_counts()


# pd.DataFrame(liste_dates)

# # On récupère uniquement les dates uniques qui ont plus de 3 occurrences
# liste_dates_filt = liste_dates[liste_dates.sort_index() > 3]
# print(liste_dates.sort_index() > 3)

# plt.figure(figsize=(20, 30))

# # On trie les résultats par ordre chronologique (donc via la fonction .sort_index())
# sns.barplot(x=liste_dates.sort_index().values, y=liste_dates.sort_index().index)

In [7]:
df["texte"] = df["texte"].apply(efface_entete).apply(retire_saut_ligne)

# df["auteur"].apply(recup_nom_auteur).apply(suppr_espace)

df["auteur"] = df["auteur"].apply(recup_nom_auteur).apply(suppr_espace)

df["sujet"] = df["sujet"].apply(suppr_espace)

# print(df["texte"])
# print(retire_saut_ligne(efface_entete(df["texte"])))

In [8]:
# Analyse TF-IDF

vectorizer = TfidfVectorizer(max_df=.65, min_df=1, stop_words=None, use_idf=True, norm=None)
transformed_documents = vectorizer.fit_transform(df["texte"].tolist())

transformed_documents_as_array = transformed_documents.toarray()

transformed_documents_as_array[0]

for counter, doc in enumerate(transformed_documents_as_array):
    # construct a dataframe
    tf_idf_tuples = list(zip(vectorizer.get_feature_names_out(), doc))
    one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True)
    
    break

In [9]:
# Export

with open("corpus4.txt", "w") as c:
    for texte, auteur, sujet in zip(df["texte"], df["auteur"], df["sujet"]):
        c.write(f"**** *auteur_{auteur} *sujet_{sujet}\n {texte}\n\n")

with open("corpus.txt", "r") as c:
    file = c.read()
    print(len(file))

2617401
